## Custom Agent Masks for subsampling the dataset

Reference: https://www.kaggle.com/fnands/makesubmasks

 - Full training set is huge.
 - For prototyping the models its better to subsample the dataset to create a smaller dataset.
 - In full training dataset, frame to frame there is not much difference in a scene. if you look at framen `n` and frame `n+1` in a scene, it will look more or less identical. This much fine grained details might be required for the final model. But in order to do faster prototyping, we can go with much more coarse grained dataset.
 - But for initial prototyping, we can subsample by skipping some consecutive frames, where will have more prominant difference frame-to-frame.


 - Also matching testset.
    - In the testset, we are asked to predict the future of the frame # 100. that is 50 future predictions.
    - So we can subsample agents with atleast 100 history frames, and 50 future frames.
    

In [1]:
import sys
from pathlib import Path

l5kit_repo = Path("../../l5kit/l5kit").resolve()
l5kit_repo

PosixPath('/home/nisarkavungal_gmx_com/l5kit/l5kit')

In [2]:
sys.path.insert(0, str(l5kit_repo)) # override the installed l5kit package

### Imports

In [3]:
import numpy as np
import pandas as pd

from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset
from l5kit.rasterization import build_rasterizer
from l5kit.data.filter import get_agents_slice_from_frames

Importing fork


In [4]:
os.environ["L5KIT_DATA_FOLDER"] = "/home/nisarkavungal_gmx_com/lyft-data"

In [7]:
dm = LocalDataManager()

ds_path = dm.require("scenes/train.zarr")
ds_path

'/home/nisarkavungal_gmx_com/lyft-data/scenes/train.zarr'

In [8]:
chunked_ds = ChunkedDataset(ds_path)
chunked_ds.open()